# CoE / NW coffee farm crawler

### 왜?
* 역대 경쟁력있는 농장들 목록을 지도에 정리하기 위해
* 연도에 따른 국가별 수상 농장들의 양상 파악
* 각 국가의 커피 경쟁력 흐름 읽기
* 특징적인 농장 커피를 쉽게 찾고 싶어서, 쇼핑링크로 이동하는 것까지
* [ACE(Alliance for Coffee Excellence) 홈페이지](https://allianceforcoffeeexcellence.org/)의 폐쇄적인 링크 구조 때문에
  농장 정보를 한눈에 얻기 어려움
    * 추가: [CoE(Cup of Excellence)](https://cupofexcellence.org/) 페이지가 동일한 데이터 가짐. 

### 수집해야 할 정보
* 파나마 CoE - ACE와 별도로 진행해야 함
* 연도별 CoE 랭킹 테이블
  - CoE와 ACE는 각각 커핑, 옥션을 담당하는 것으로 보임

In [2]:
import requests
from urllib.parse import urlsplit, urlunsplit
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
class Config():
    ace_url = 'https://allianceforcoffeeexcellence.org/'
    coe_url = 'https://cupofexcellence.org/'
    coe_url_components = urlsplit(coe_url)
    coe_results_url = "https://cupofexcellence.org/competition-auction-results/"
#     nw_url = "https://allianceforcoffeeexcellence.org/nw-competition-auction-results/"
#     excludes = {
#         "https://allianceforcoffeeexcellence.org/farm-directory/",
#     }
#     coe_nw_text = "COE and National Winner Results"

### URL 정규화
National Winner 링크의 경우 base url이 생략된 경우가 섞여 있다.

In [4]:
def normalize_url(url):
    url_components = urlsplit(url)
    url_components = url_components\
        ._replace(scheme = Config.coe_url_components.scheme)\
        ._replace(netloc = Config.coe_url_components.netloc)
    clean_url = urlunsplit(url_components)
    return clean_url

In [5]:
def acquire_coe_list():
    response = requests.get(Config.coe_results_url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        menu = soup.find('ul', id='menu-coe-country-programs-menu')
        links = filter(
            lambda li: \
                'menu-item-has-children' not in li['class'], #and \
                #li.a['href'] not in Config.excludes,
            menu.findAll('li', 'menu-item')
        )
        return map(lambda li: normalize_url(li.a['href']), links)

    else: 
        print(response.status_code)
        return
    
#list(acquire_coe_list())

In [6]:
def crawl_farm_table_container(url):
    response = requests.get(url)
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        container = soup.find('div', 'vc_tta-container')
        return container
    else: 
        print(response.status_code)
        return

#crawl_farm_table_container("https://cupofexcellence.org/brazil-naturals-2015-january/")

In [7]:
coe_url_list = list(acquire_coe_list())
coe_table_container_list = []

In [8]:
for index, url in enumerate(coe_url_list[len(coe_table_container_list):]):
    tables = crawl_farm_table_container(url)
    coe_table_container_list.append(tables)
    print(f"{index}: {url}")

0: https://cupofexcellence.org/bolivia-2009/
1: https://cupofexcellence.org/bolivia-2008/
2: https://cupofexcellence.org/bolivia-2007/
3: https://cupofexcellence.org/bolivia-2005/
4: https://cupofexcellence.org/bolivia-2004/
5: https://cupofexcellence.org/brazil-2021/
6: https://cupofexcellence.org/brazil-2020/
7: https://cupofexcellence.org/brazil-2019/
8: https://cupofexcellence.org/brazil-naturals-2018/
9: https://cupofexcellence.org/brazil-naturals-2017/
10: https://cupofexcellence.org/brazil-naturals-2016/
11: https://cupofexcellence.org/brazil-naturals-2015/
12: https://cupofexcellence.org/brazil-naturals-2015-january/
13: https://cupofexcellence.org/brazil-naturals-2014/
14: https://cupofexcellence.org/brazil-naturals-2013/
15: https://cupofexcellence.org/brazil-naturals-2012/
16: https://cupofexcellence.org/brazil-pulped-naturals-2018/
17: https://cupofexcellence.org/brazil-pulped-naturals-2017/
18: https://cupofexcellence.org/brazil-pulped-naturals-2016/
19: https://cupofexcel

163: https://cupofexcellence.org/peru-2021/
164: https://cupofexcellence.org/peru-2020/
165: https://cupofexcellence.org/peru-2019/
166: https://cupofexcellence.org/peru-2018/
167: https://cupofexcellence.org/peru-2017/


### Check integrity of `vc_tta-container`

(ACE 기준)

* CoE 2019에서는 `Winning Farms*` 탭에 기록
* CoE 2020에서는 `CoE Competition Results`가 없음(`COE Auction Results`만 존재)
* CoE 2021에서는 `Winning Farms`가 아닌 `COE Competition Results` 탭에 기록
* 이런 개 그지같은 
* 아래 not in을 스위치하면서 확인해볼 것

(CoE)
* 추가: CoE competition Results, CoE Auction Results는 CoE페이지에서 Winning Farms로 나옴

In [28]:
tables_name_list = []
coe_table_name = ["Winning Farms", "Winning Farms*"]
#nw_table_name = [""]

def tab_integrity_test():
    for index, table_container in enumerate(coe_table_container_list):
        table_name_list = list(map(lambda li: li.string, ul.findAll('li')))
        tables_name_list.append(table_name_list)
        #print(len(set(table_name_list) & set(farms_tab_names)))
        if "Winning Farms" not in table_name_list:
            if "Winning Farms*" not in table_name_list:
            #    if "COE Competition Results" not in table_name_list:
                print(coe_url_list[index], coe_table_name)
                raise Exception("Cannot find tab name 'Winning Farms'")

tab_integrity_test()

In [31]:
def get_table_lists_from_container(container):
    return container.findAll('table')

tables_list = list(map(get_table_lists_from_container, coe_table_container_list))

### table head, body 추출
* 브라질 통합 링크의 경우 테이블에 `thead` 태그가 없다

In [39]:
def has_th(table):
    thead_elements = table.findAll('th')
    if thead_elements == []:
        return False
    return True

def select_link(td_list):
    links = list(filter(lambda x: x, map(lambda td: td.find('a'), td_list)))
    if not links:
        print(td_list)
    return links

def map_string_to_row(row):
    return list(map(lambda td: td.text.strip(), row))

def get_thead_from_table(table):
    if has_th(table):
        thead_elements = table.findAll('th')
    else:
        thead_elements = table.find('tr').findAll('td')
    thead_list = list(map(lambda th: th.text, thead_elements))
    return thead_list

def get_tbody_from_table(table):
    if has_th(table):
        tbody_list = table.findAll('tr')
    else:
        tbody_list = table.findAll('tr')[1:]
    tbody_list = list(map(lambda tr: tr.findAll('td'), tbody_list))
    tbody_list = list(map(map_string_to_row, tbody_list))
    return tbody_list

def get_farm_from_table(table):
    if has_th(table):
        tbody_list = table.findAll('tr')
    else:
        tbody_list = table.findAll('tr')[1:]
    tbody_list = list(map(lambda tr: tr.findAll('td'), tbody_list))
    tbody_list = list(map(select_link, tbody_list))
    return tbody_list

def get_theads_from_tables(tables_list):
    theads_list = []
    for i, table_list in enumerate(tables_list):
        thead_list = list(map(get_thead_from_table, table_list))
        theads_list.append(thead_list)
    return theads_list

def get_tbodys_from_tables(tables_list):
    tbodys_list = []
    for i, table_list in enumerate(tables_list):
        tbody_list = list(map(get_tbody_from_table, table_list))
        tbodys_list.append(tbody_list)
    return tbodys_list

def get_farms_from_tables(tables_list):
    farms_list = []
    for i, table_list in enumerate(tables_list):
        farm_list = list(map(get_farm_from_table, table_list))
        farms_list.append(farm_list)
    return farm_list



### label 비교

* 봐라 라벨이 다 따로논다, 이거 맞추다가 눈 빠지겠다

In [13]:
label_map = {

    'Coffee Washing Station Name': 'Farm / CWS',
    'FARM':        'Farm / CWS',
    'FARM / CWS':  'Farm / CWS',
    'FARM/CWS':    'Farm / CWS',
    'Farm':        'Farm / CWS',
    'Farm / CWS':  'Farm / CWS',
    
    'FARMER':                    'Farmer / Representative',
    'FARMER / REPRESENTATIVE':   'Farmer / Representative',
    'Farmer':                    'Farmer / Representative',
    'Farmer / Representative':   'Farmer / Representative',
    'FARMER / ORGANIZATION':     'Farmer / Representative',
    
    'Company Name': None,
    'PRODUCER':     'Producer',
    'Producer':     'Producer',
    'Coffee Washing Station Owner': None,
    'OWNER':        'Owner',
    
    'High Bid': None,
    'High Bidder(s)': None,
    'Total Value': None,

    'RANK': 'Rank',
    'Rank': 'Rank',

    'SCORE': 'Score',
    'Score': 'Score',

    'PROCESS': 'Process',
    'PROCESSING': 'Process',
    'Process': 'Process',
    'Processing': 'Process',

    'VARIETY': 'Variety',
    'Variety': 'Variety',
    
    'REGION': 'Region',
    'Region': 'Region',
    
    'LOT NO.':           "Size",
    'SIZE':              "Size",
    'SIZE (30KG BOXES)': "Size",
    'Size':              "Size",
    'Size (30kg Boxes)': "Size",
    'WEIGHT (kg)': 'Weight',
    'Weight (kg)': 'Weight',
    'Weight (lbs)': 'Weight',
    'Weight (lbs.)': 'Weight',
    
    'Woreda': 'Woreda',
    'WOREDA': 'Woreda',
    'Zone': 'Zone',
    'ZONE': 'Zone',
}

def regularize_cols(thead):
    return list(map(lambda i: label_map[i], thead))

In [14]:
def info_from_url(url):
    # brazil-naturals-2015-january
    url = url.replace('2015-january', '2015_january')
    # costa-rica-coe
    url = url.replace('costa-rica-coe', 'costa-rica')
    suffix = urlsplit(url.title()).path.strip('/')
    return suffix.replace("-", ' ').rsplit(' ', 1)

for url in coe_url_list: print(info_from_url(url))

['Bolivia', '2009']
['Bolivia', '2008']
['Bolivia', '2007']
['Bolivia', '2005']
['Bolivia', '2004']
['Brazil', '2021']
['Brazil', '2020']
['Brazil', '2019']
['Brazil Naturals', '2018']
['Brazil Naturals', '2017']
['Brazil Naturals', '2016']
['Brazil Naturals', '2015']
['Brazil Naturals', '2015_January']
['Brazil Naturals', '2014']
['Brazil Naturals', '2013']
['Brazil Naturals', '2012']
['Brazil Pulped Naturals', '2018']
['Brazil Pulped Naturals', '2017']
['Brazil Pulped Naturals', '2016']
['Brazil Pulped Naturals', '2015']
['Brazil Pulped Naturals', '2014']
['Brazil Pulped Naturals', '2013']
['Brazil Pulped Naturals', '2012']
['Brazil Pulped Naturals', '2011']
['Brazil Pulped Naturals', '2011']
['Brazil Pulped Naturals', '2009']
['Brazil Pulped Naturals', '2008']
['Brazil Pulped Naturals', '2006']
['Brazil Pulped Naturals', '2005']
['Brazil Pulped Naturals', '2004']
['Brazil Pulped Naturals', '2003']
['Brazil Pulped Naturals', '2002']
['Brazil Pulped Naturals', '2001']
['Brazil Pulped 

In [15]:
D = []
for i, url in enumerate(coe_url_list):
    tnl = tables_name_list[i]
    if "Winning Farms" in tnl:
        j = tnl.index("Winning Farms")
    else:
        j = tnl.index('Winning Farms*')
    thead, tbody = theads_list[i][j], tbodys_list[i][j]
    for l in tbody:
        if l == []: continue         # brundi 2018 error
        d = {k: v for k, v in zip(regularize_cols(thead), l)}
        d['Country'], d['Year'] = info_from_url(url)
        if d['Rank'] != '':
            D.append(d)

In [16]:
for d in D:   
    # Guatemala 2001
    if 'Score' not in d: 
        d['Score'] = -1      
    else:
        d['Score'] = float(d['Score'])
        #if d['Score'] < 80.0: print(d)
        
    # Ethiopia 2020
    if 'Region' not in d: 
        d['Region'] = f"{d['Woreda']}, {d['Zone']}"
        del d['Woreda'], d['Zone']

    # Ethiopia 2021
    if 'Woreda' in d:
        d['Region'] = f"{d['Woreda']}, {d['Zone']}, {d['Region']}"
        del d['Woreda'], d['Zone']

    # Colombia - producer
    if 'Producer' in d:
        d['Farmer / Representative'] = d['Producer']
        del d['Producer']
    
    # Burundi 2019 - owner
    if "Owner" in d and "Farmer / Representative" not in d:
        d["Farmer / Representative"] = d['Owner']
        del d['Owner']
    

    # Ethiopia 2020, 2021
    if 'Farm / CWS' not in d:
        d['Farm / CWS'] = "N/A"
    
    assert 'Rank' in d
    assert 'Region' in d
    assert 'Score' in d
    assert 'Farm / CWS' in d
    assert 'Farmer / Representative' in d

D = list(filter(lambda d: d['Score'] > 80.0, D))

In [17]:
df = pd.DataFrame.from_dict(D)
df

,Rank,Size,Farm / CWS,Farmer / Representative,Region,Score,Country,Year,Variety,Process,Weight
0,1,24,Agrotakesi SA,Mauricio Ramiro Diez de Medina,"Yanacahi, Yungas of La Paz",93.36,Bolivia,2009,NaN,NaN,NaN
1,2,28,Café Sima del Jaguar A,Braulio Luque Yana,"Caranavi, Yungas of La Paz",92.05,Bolivia,2009,NaN,NaN,NaN
2,3,23,Café Monterrey,Valentin Choquehuanca Aduviri,"Caranavi, Yungas of La Paz",91.62,Bolivia,2009,NaN,NaN,NaN
3,4,20,Café Jacaranda,Cruz Elias Choconapi,"Caranavi, Yungas of La Paz",91.38,Bolivia,2009,NaN,NaN,NaN
4,5,21,Café Alan Coffe,Luis Yujra Arismende,"Caranavi, Yungas of La Paz",90.90,Bolivia,2009,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4459,15,11,Las Pircas 2017,Nerio Collantes Herrera,"Moyobamba, San Martin",87.09,Peru,2017,NaN,NaN,NaN
4460,16,11,El Shimir 2017 – {RAO},Marvin Charles Ramos Chinc,"San Ignacio, Cajamarca",86.55,Peru,2017,NaN,NaN,NaN
4461,17,14,Santa Sofia 2017,Eluterio Bolaños Torrecillas,"Satipo, Junin",86.34,Peru,2017,NaN,NaN,NaN
4462,18,30,Monte Rey 2017,Felix Cachay Cueva,"Chanchamayo, Junin",86.16,Peru,2017,NaN,NaN,NaN


In [18]:
df.to_json("./coe_list.json")